# Problem description

This notebook illustrates a CPLEX MIP model and a CP Optimizer mode to solve a single-machine scheduling problem.

# Reading the data

In [1]:
#Duration = [2,4,1,1,1,2]
#Precedences = [[0,2], [0,3], [2,4], [3,4], [3,5]]

Duration = [2,4,1,1,1,2,2,4,5,6,2,4,1,8,4,6,3,1,5,6,3,8]
Precedences = [[0,2], [0,3], [2,4], [3,4], [3,5], [3,11], [4,17],[11,18],[7,9],[16,18]]

n = len(Duration)

# Modeling the problem with CP Optimizer

In [2]:
# Import Constraint Programming modelization functions
from docplex.cp.model import *

# Create model object
model = CpoModel()

# Decision variables
jobs = [ interval_var(size=Duration[i]) for i in range(n)]

# Objective function
model.add(minimize(sum(end_of(jobs[i]) for i in range(n))))

#f = sum(pulse(jobs[i],1) for i in range(n))

# Constraints: jobs do not overlap on the machine
model.add(no_overlap(jobs))
#model.add(f <= 2)

# Constraints: precedence constraints
for p in Precedences:
    model.add(end_before_start(jobs[p[0]],jobs[p[1]]))

# Solving the problem with CP Optimizer automatic search

The model can be solved by calling CP Optimizer's automatic search:

In [3]:
# Solve the model
sol = model.solve(trace_log=True,LogPeriod=1000000000,TimeLimit=30)

 ! ----------------------------------------------------------------------------
 ! Minimization problem - 23 variables, 11 constraints
 ! TimeLimit            = 30
 ! LogPeriod            = 1000000000
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 98.1 (before), 98.1 (after)
 !  . Memory usage      : 443.6 kB (before), 443.6 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0         23                 -
 + New bound is 107
 ! Using iterative diving.
 *           964       45  0.07s        1      (gap is 88.90%)
 *           681      137  0.07s        1      (gap is 84.29%)
 *           616      181  0.07s        1      (gap is 82.63%)
             616     2423          2    1   F        -
 + New bound is 176 (gap is 71.43%)
             616     2423          2    1    

# Displaying the solution

In [ ]:
%matplotlib notebook
import docplex.cp.utils_visu as visu
if sol:
    visu.panel("Jobs")
    for i in range(n):
        visu.interval(sol.get_var_solution(jobs[i]), i, str(i))
    visu.show()